# Connect NN Model with Arduino

The purpose of this notebook is to process an audio file, classify it with the NN model, then send the resulting command to the arudino car to excecute.

#### Function to Send Command to Arduino

In [31]:
import serial
import time
# your Serial port should be different!
#arduino = serial.Serial('/dev/cu.HC-06-SPPDev', 9600)

def onOffFunction(command):
    #arduino = serial.Serial('/dev/cu.HC-06-SPPDev', 9600)
    #command = input("type go, stop, left, or right")
    if command =="go":
        #print("Forward")
        #time.sleep(.01) 
        arduino.write(str.encode('F')) 
        #onOffFunction()
    elif command =="stop":
        #print("Stop")
        #time.sleep(.01) 
        arduino.write(str.encode('S'))
        #onOffFunction()
    elif command =="left":
        #print("Left")
        #time.sleep(.01) 
        arduino.write(str.encode('L'))
        #onOffFunction()
    elif command =="right":
        #print("Right")
        #time.sleep(.01) 
        arduino.write(str.encode('R'))
        #onOffFunction()
    elif command =="two":
        #print("Right")
        #time.sleep(.01) 
        arduino.write(str.encode('T'))
        #onOffFunction()
    elif command =="three":
        #print("Right")
        #time.sleep(.01) 
        arduino.write(str.encode('E'))
        #onOffFunction()
    elif command =="happy":
        #print("Right")
        #time.sleep(.01) 
        arduino.write(str.encode('H'))
        #onOffFunction()
    else:
        print("No command recognized!")

#time.sleep(2) #waiting the initialization...

#onOffFunction()

#### Load NN model

In [32]:
from keras.models import model_from_json
#load json and create model
json_file = open('model_car_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_car_final.h5")
print("Loaded model from disk")

Loaded model from disk


#### Functions to process sounds

In [33]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield int(start), int(start + window_size)
        start += (window_size / 2)

def process_sounds(signal,bands):
    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
    logspec = librosa.power_to_db(melspec, ref=np.max)
    logspec = logspec.T.flatten()[:, np.newaxis].T
    
    return logspec
        

def extract_features(fn,bands = 128, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    
        
    sound_clip,s = librosa.load(fn)

    for (start,end) in windows(sound_clip,window_size):
        if(len(sound_clip[start:end]) == window_size):
            signal = sound_clip[start:end]
            logspec = process_sounds(signal,bands)
            log_specgrams.append(logspec)
            #mfccs.append(logmfcc)
            #labels.append(label)
    
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    
    
    
    return np.array(log_specgrams)

#### Function to get sound, process it, and run through model

In [34]:
def model_output(sample):
    test_sample = extract_features(sample)
    print(test_sample.shape)
    test_sample  = np.pad(test_sample, [(0, 0),(0, 0),(0, 7),(0, 2)], mode='constant', constant_values=0)
    test_pred = loaded_model.predict(test_sample)
    
    return test_pred

In [19]:
def find_cmd(pred):
    ind = np.argmax(pred)
    cmd = "No command recognized"
    if pred[ind]>.92:
        cmd = cmds[ind]
    return cmd

In [35]:
import pyaudio
import wave
import audioop
from collections import deque
import os
import time
import math
import librosa
import numpy as np

LANG_CODE = 'en-US'  # Language to use

GOOGLE_SPEECH_URL = 'https://www.google.com/speech-api/v1/recognize?xjerr=1&client=chromium&pfilter=2&lang=%s&maxresults=6' % (LANG_CODE)

FLAC_CONV = 'flac -f'  # We need a WAV to FLAC converter. flac is available
                       # on Linux

# Microphone stream config.
CHUNK = 1024  # CHUNKS of bytes to read each time from mic
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
THRESHOLD = 1200  # The threshold intensity that defines silence
                  # and noise signal (an int. lower than THRESHOLD is silence).

SILENCE_LIMIT = 1  # Silence limit in seconds. The max ammount of seconds where
                   # only silence is recorded. When this time passes the
                   # recording finishes and the file is delivered.

PREV_AUDIO = .5  # Previous audio (in seconds) to prepend. When noise
                  # is detected, how much of previously recorded audio is
                  # prepended. This helps to prevent chopping the beggining
                  # of the phrase.
time_len=.5
def listen_for_speech(threshold=THRESHOLD, num_phrases=-1):
    """
    Listens to Microphone, extracts phrases from it and sends it to 
    Google's TTS service and returns response. a "phrase" is sound 
    surrounded by silence (according to threshold). num_phrases controls
    how many phrases to process before finishing the listening process 
    (-1 for infinite). 
    """

    #Open stream
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* Listening mic. ")
    audio2send = []
    cur_data = ''  # current chunk  of audio data
    rel = int(RATE/CHUNK)
    slid_win = deque(maxlen=SILENCE_LIMIT * rel)
    #Prepend audio from 0.5 seconds before noise was detected
    prev_audio = deque(maxlen=int(PREV_AUDIO * rel))
    started = False
    n = num_phrases
    response = []

    
    
    while (num_phrases == -1 or n > 0):
        cur_data = stream.read(CHUNK,exception_on_overflow = False)
        slid_win.append(math.sqrt(abs(audioop.avg(cur_data, 4))))
        #print slid_win[-1]
        
        tsum = sum([x > THRESHOLD for x in slid_win])
        
        if tsum > 0:
            for i in range(0, int(RATE / CHUNK * time_len)):
                data = stream.read(CHUNK)
                audio2send.append(data)
            slid_win = deque(maxlen=SILENCE_LIMIT * rel)
            started=True
        elif started == True:
            # The limit was reached, finish capture and deliver.
            
            preds, filename = save_speech(list(prev_audio) +audio2send, p)
            cmds = ['go','happy','left','right','stop','three','two']
            cmd = 'No command recognized'
            #print(preds)
            #vfunc = np.vectorize(find_cmd)
            #cmd = vfunc(preds)
            for pred in preds:
                ind = np.argmax(pred)
                if pred[ind]>.92:
                    cmd = cmds[ind]
            print(cmd)
            if cmd == 'three':
                break
            onOffFunction(cmd)
            
            # Send file to Model and get response
            # Remove temp file. Comment line to review.
            # Reset all
            started = False
            slid_win = deque(maxlen=int(SILENCE_LIMIT * rel))
            prev_audio = deque(maxlen=1 * rel)
            audio2send = []
            n -= 1
            
            print ("Listening ...")
        else:
            prev_audio.append(cur_data)

    print("* Done recording")
    stream.close()
    p.terminate()

    return 'done'


def save_speech(data, p):
    """ Saves mic data to temporary WAV file. Returns filename of saved 
        file """
    
    filename = 'Junk/output_'+str(int(time.time()))
    # writes data to WAV file
    data = b''.join(data)
    wf = wave.open(filename + '.wav', 'wb')
    wf.setnchannels(2)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(RATE)  # TODO make this value a function parameter?
    wf.writeframes(data)
    wf.close()
    preds = model_output(filename + '.wav')
    #print(preds)
    return (preds,filename)

## Excecute Program

In [36]:
arduino = serial.Serial('/dev/cu.HC-06-SPPDev', 9600)
listen_for_speech()

* Listening mic. 


KeyboardInterrupt: 